In [20]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import time
from riotwatcher import LolWatcher, ApiError
from sklearn.datasets import load_iris
import plotly.express as px

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

In [21]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

In [22]:
ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

In [107]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = pd.concat([champions, pd.json_normalize(champions.data)], axis=1).drop(
    columns=["type", "format", "version", "data", "name", "id"]
)

champions.insert(0, "key", champions.pop("key"))
champions[["primary_role", "secondary_role"]] = pd.DataFrame(
    champions.tags.tolist(), index=champions.index
)
champions = champions.drop(columns="tags")
champions.primary_role = champions.primary_role.astype("category")
champions.secondary_role = champions.secondary_role.astype("category")
champions.head()

,key,champion,title,blurb,partype,info.attack,info.defense,info.magic,info.difficulty,image.full,...,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.00,2.500,0.651,Fighter,Tank
1,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,3,4,8,5,Ahri.png,...,8.0,0.8,0,0,53,3.00,2.000,0.668,Mage,Assassin
2,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,5,3,8,7,Akali.png,...,50.0,0.0,0,0,62,3.30,3.200,0.625,Assassin,NaN
3,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,0,0,0,0,Akshan.png,...,8.2,0.7,0,0,52,3.50,4.000,0.638,Marksman,Assassin
4,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,6,9,5,7,Alistar.png,...,8.5,0.8,0,0,62,3.75,2.125,0.625,Tank,Support


In [25]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("NA1", "yogurtsauce")

In [26]:
Summoner(NA, "yogurtsauce").matches_

['NA1_4651135323',
 'NA1_4651119139',
 'NA1_4651100905',
 'NA1_4651072722',
 'NA1_4651069771',
 'NA1_4650032208',
 'NA1_4650015207',
 'NA1_4649897302',
 'NA1_4649867811',
 'NA1_4649806006',
 'NA1_4649782939',
 'NA1_4649388982',
 'NA1_4649373402',
 'NA1_4649362036',
 'NA1_4648877961',
 'NA1_4648853094',
 'NA1_4648831552',
 'NA1_4648478444',
 'NA1_4648469619',
 'NA1_4648465593']

In [27]:
j = 0
cont = 0
last_match = yogurtsauce.matches_[0]
match_detail = lol_watcher.match.by_id(NA, last_match)

In [28]:
match_detail

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4651135323',
  'participants': ['dQTyaKFTNAj0uYrfqOW-II9cVIIOUKGkZ_sbblL0tmJXjWjjCuhSwkEZBUJBM73kkWJaPuXT8OqmcQ',
   '0_Z8M5TjaB96qj5ncZ3ewtq3H6CrLW6znsMYjxQ0Cgcrjr5N72FCnBVahB024J_o0biXH3Vi1TyBqw',
   'YGKig706lJAJIFb2SiGT6AMzW_Pp8ma_0gWm5kt0wiU_y7dSGjwty6Gcqkj8Ry0iGMi4QfkxwOWDgw',
   'GattObQ5NkBPysjXcwtBZxLP3HDBRICqNxjBW98fnO78B4a_DIRY_BAx1Gia-dnoXvE7LqAzPiDQQQ',
   'HoWKxMmDt5D_RZHfLWyae6esTHIXBK55I0PbWoXG7Ramh5WEfEzR00PQlq8WYOjtaP0z47OzRwH9Lg',
   'LXG414RLMjXx1Ivn36hW8Io9ygwdxfyRuwMjOLTkjPRCCyRufBQ0fPXpIVHeljACJe2YnrmtDd1_lA',
   'eSXsdXu5Bsg2eKq4vxBeCu0U3aavHDWKiG767yYd6WwripxRLpwPSW_TZBgQuoS43p6gzfY2XMvnBg',
   'k_gYZNyf-IJVxLd-YJ5uawj9LYdfeuXjY4zDd_Sb6VKg3fW1ACJCO--cOUgSx6-7SOsSKYt3jhNKYw',
   '71mVFP73L1Xm7OscCrfPUcYtsl2W3jBG05HTVwh8A_7a01ZM9DHVt0YVxf3QOdDa9hZDcu9RFMc9-Q',
   'zLFXTakGw16adLrcdutODOoniKYaMtdJQ-qvTKF5U3zosg0dOUMqvPBOwbxKmMGJFsCxYDwYVeyKQA']},
 'info': {'gameCreation': 1683613809314,
  'gameDuration': 1078,
  

In [29]:
lol_watcher.match.by_id(NA, "NA1_4648853094")

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4648853094',
  'participants': ['jUi2m-kG_zgD1pkRLMNV7jxZxxaRA4TT3ot6QIkHTG5Ro79DcoUFJUGXE0-s2CQsGks0__GdiYr3Dg',
   'tnif9nKmmXcDE09sq-UFRTks08oLNchpsORl0OcN1PSw2-ccfBkLHDm4_0kFhZegNhSyVVoMQhndoQ',
   'e77O5n1RHHste-rZx3E9i3BAwEAih0OZGvaHDepGUmDqjOVZ1pqAchbReltEMzKmBltG5c40sbt-Jw',
   '3bE1YJHfpAvhG1qeoRl9qMZlmJNuBl9XCwCSeqWo3WImUaqVk_5eULQkDJxVkxrU0CvIhR509FiOkA',
   'dAwBLRp6hsERYCRZMSb8jwpE6P4YR6HEIa_3cnkdd8wyjG48Gczw33Ry4_676egSywTt6PL6tVOrqw',
   'qnMGsVMYZprXw5xAjC4Yz_UgnPsKXgoDEsz6CSXa64BrbeAPI5oZwqWN1niSlHy-DEM0wCAsvO4KDw',
   '5F4QVVM_-MMmPZK8yk_Ret-Y-WoE9awsUdp5sGceWNq2Lau4pJsrG_wv0ZbDnMEUnmAp8gWLlMF64g',
   'nZT4_77dyUdjarx3Gbyu2N0CLeDcGP78z2FWhgOSdBsNjvnUx1aR-nuuqyfTcIa7qrAhHtPMjKUMNQ',
   '0_Z8M5TjaB96qj5ncZ3ewtq3H6CrLW6znsMYjxQ0Cgcrjr5N72FCnBVahB024J_o0biXH3Vi1TyBqw',
   'O-R1EGpWaAUCFkw6UPItBg3SwEXxSkJTck-0lsRgveKxwW74j9eUCDjKeQaKfcpvtj54N8gAMLlhqw']},
 'info': {'gameCreation': 1683414599891,
  'gameDuration': 1217,
  

In [119]:
N_GAMES = 5

games = {}
game_duration = np.zeros(N_GAMES)
damage = np.zeros(N_GAMES)

j = 0
cont = 0

while cont < N_GAMES:
    try:
        last_match = yogurtsauce.matches_[cont]
        match_detail = lol_watcher.match.by_id(NA, last_match)
        participants = []

        for row in match_detail["info"]["participants"]:
            temp = {}
            temp["summoner_id"] = row["summonerId"]
            temp["name"] = row["summonerName"]
            temp["level"] = row["summonerLevel"]
            temp["champion_id"] = row["championId"]
            temp["champion"] = row["championName"]
            temp[
                "mastery_points"
            ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                NA, temp["summoner_id"], temp["champion_id"]
            )[
                "championPoints"
            ]
            temp["team"] = row["teamId"]
            temp["win"] = row["win"]


            participants.append(temp)

        games[j] = pd.DataFrame(participants)
        games[j].win = games[j].win.astype("category")

        j += 1
        cont += 1
    except:
        cont += 1

In [120]:
games[0]

,summoner_id,name,level,champion_id,champion,mastery_points,team,win
0,HPfznbOF0b1uI_hSHt-TrLgLFQGc_Eo821g2yFaR93WNJbY,Reasonably Salty,163,41,Gangplank,29463,100,False
1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q,yogurtsauce,512,516,Ornn,23104,100,False
2,0KE8TfqbxIlyQI9tnG0HlNm4oAM7eIuxPdYQ3GkxNUUsba8,Flydaze,101,53,Blitzcrank,4315,100,False
3,GV0q3LeAsQQUWT6p9kK_wWIe_urOc0Wz0rGyr8vnTlChFQo,Ogrenix,54,61,Orianna,1793,100,False
4,DXYtTgjUFMEBqBUjrgEJhq7qQdVNSbwtk7_rfS7gupNMg3o,Shuzzz,130,74,Heimerdinger,11655,100,False
5,xoFD4796dIDl69D0pMSP8mJCaZPYYOFmbeD4Xr4ccof8OAY,Franky Jayy,382,13,Ryze,49023,200,True
6,ZBsFREw02R4Ti0nOFR7YifcncsUMR-V-XHsHBuPqEeN3hAU,AyDrin,233,86,Garen,37449,200,True
7,Bzb0W8QepevYMfjkdUJy8g-8Ql3NzSnMXLiliIVxLFd-GWc,HaronTheWicked,325,5,XinZhao,56852,200,True
8,XK3fNvGo44PfyeRFlvMUqXYdVp0x5G-Ka4rFlPvVoxWku54,ßepsı,375,101,Xerath,34626,200,True
9,JGtmidMxeyymthNRfZbNE3h5OuQCfTy9C1aTz0_5Rc93Ocw,KASPALL,355,235,Senna,35701,200,True


In [117]:
champions.columns

Index(['key', 'champion', 'title', 'blurb', 'partype', 'info.attack',
       'info.defense', 'info.magic', 'info.difficulty', 'image.full',
       'image.sprite', 'image.group', 'image.x', 'image.y', 'image.w',
       'image.h', 'stats.hp', 'stats.hpperlevel', 'stats.mp',
       'stats.mpperlevel', 'stats.movespeed', 'stats.armor',
       'stats.armorperlevel', 'stats.spellblock', 'stats.spellblockperlevel',
       'stats.attackrange', 'stats.hpregen', 'stats.hpregenperlevel',
       'stats.mpregen', 'stats.mpregenperlevel', 'stats.crit',
       'stats.critperlevel', 'stats.attackdamage',
       'stats.attackdamageperlevel', 'stats.attackspeedperlevel',
       'stats.attackspeed', 'primary_role', 'secondary_role'],
      dtype='object')

In [118]:
games[1].merge(
    champions[
        [
            "champion",
            "primary_role",
            "secondary_role",
            "info.attack",
            "info.defense",
            "info.magic",
            "info.difficulty",
        ]
    ]
)

,summoner_id,name,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty
0,Vjv5QDnIO0t1c5H5mo4ZROoLtpGoXKo9h7Zv8r6gJlWKXy4,OhBoyAnotherAcc,207,29,Twitch,19266,100,True,Marksman,Assassin,9,2,3,6
1,CM7rzQCpuL575RAmaNG3ojZoGohrMxRmnVvWgNRGR44zB5Q,ThrKen,153,86,Garen,15350,100,True,Fighter,Tank,7,7,1,5
2,qrivDaeifv-ChIZlXlzXZxC0trXHuoQi3jAy4zlM18x7SA8,TheCrocodileKing,341,11,MasterYi,27391,100,True,Assassin,Fighter,10,4,2,4
3,HPfznbOF0b1uI_hSHt-TrLgLFQGc_Eo821g2yFaR93WNJbY,Reasonably Salty,163,10,Kayle,19153,100,True,Fighter,Support,6,6,7,7
4,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q,yogurtsauce,512,82,Mordekaiser,13145,100,True,Fighter,NaN,4,6,7,4
5,JqB0YoiSitiZ3m1dRCzQxLpq--kb_GE6eEprMGzUT5rywt0,NinjaCat527,250,76,Nidalee,27022,200,False,Assassin,Mage,5,4,7,8
6,x8-bum8W_RBKgUi5m_l4o2pFvy2CIp12GnNbfbU41FDyY_M,Tallennar,418,50,Swain,28553,200,False,Mage,Fighter,2,6,9,8
7,5g7LaW5Je42YsNv_BQ0qSDFNtczbIwmD7qWjsxHZF0NO7eo,Eren T Yeager,230,63,Brand,6561,200,False,Mage,NaN,2,2,9,4
8,1JQzwfJ7VXmbd_ZBjzT4V7fCH5vrDScFafPhTtwFUGq6cMEd,ß1ªckGX,545,157,Yasuo,243385,200,False,Fighter,Assassin,8,4,4,10
9,1Muumv5FMvHcK6Gcr9yASc4u9866iKncXCas0rG77crcx8Y,Pønt,566,35,Shaco,20854,200,False,Assassin,NaN,8,4,6,9
